### Exercise TO DO
- Take the code from day1 and incorporate it here, to build a website summarizer that uses Llama 3.2 running locally instead of OpenAI; use either of the approaches learned in day 2.

### EXERCISE SOLUTION

Upgrade the day 1 project to summarize a webpage to use an Open Source model running locally via Ollama rather than OpenAI

You'll be able to use this technique for all subsequent projects if you'd prefer not to use paid APIs.

**Benefits**:

- No API charges - open-source
- Data doesn't leave your box

**Disadvantages**:

- Significantly less power than Frontier Model

In [14]:
# imports required libraries

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

In [15]:
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

In [16]:
# To give you a preview -- calling OpenAI with these messages is this easy. Any problems, head over to the Troubleshooting notebook.

message = "Hello, Llama! This is my first ever message to you! Hi!"
response = ollama_via_openai.chat.completions.create(model="llama3.2", messages=[{"role":"user", "content":message}])
print(response.choices[0].message.content)

*virtual handshake* Welcome! I'm thrilled to meet you for the very first time! I'm here to help answer any questions, provide information, or just chat with you. How's your day going so far? Is there something specific you'd like to talk about or ask about, or do you just want to say hello and get to know me a bit?


In [17]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [28]:
website_url="https://timesofindia.indiatimes.com/business/india-business/busy-day-at-white-house-25-tariff-on-india-dead-economies-jibe-trade-deal-with-pakistan-more-key-points/articleshow/123011589.cms"

In [ ]:
client = Website(website_url)
print(client.title)
print(client.text)

'Communities' of strange, extreme life seen for first time in deep ocean
Skip to content
British Broadcasting Corporation
Home
News
Sport
Business
Innovation
Culture
Arts
Travel
Earth
Audio
Video
Live
Home
News
Israel-Gaza War
War in Ukraine
US & Canada
UK
UK Politics
England
N. Ireland
N. Ireland Politics
Scotland
Scotland Politics
Wales
Wales Politics
Africa
Asia
China
India
Australia
Europe
Latin America
Middle East
In Pictures
BBC InDepth
BBC Verify
Sport
Business
Executive Lounge
Technology of Business
Future of Business
Innovation
Technology
Science & Health
Artificial Intelligence
AI v the Mind
Culture
Film & TV
Music
Art & Design
Style
Books
Entertainment News
Arts
Arts in Motion
Travel
Destinations
Africa
Antarctica
Asia
Australia and Pacific
Caribbean & Bermuda
Central America
Europe
Middle East
North America
South America
World’s Table
Culture & Experiences
Adventures
The SpeciaList
Earth
Natural Wonders
Weather & Science
Climate Solutions
Sustainable Business
Green Living
A

In [19]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [20]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [21]:
print(user_prompt_for(client))

You are looking at a website titled 'Communities' of strange, extreme life seen for first time in deep ocean
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

Skip to content
British Broadcasting Corporation
Home
News
Sport
Business
Innovation
Culture
Arts
Travel
Earth
Audio
Video
Live
Home
News
Israel-Gaza War
War in Ukraine
US & Canada
UK
UK Politics
England
N. Ireland
N. Ireland Politics
Scotland
Scotland Politics
Wales
Wales Politics
Africa
Asia
China
India
Australia
Europe
Latin America
Middle East
In Pictures
BBC InDepth
BBC Verify
Sport
Business
Executive Lounge
Technology of Business
Future of Business
Innovation
Technology
Science & Health
Artificial Intelligence
AI v the Mind
Culture
Film & TV
Music
Art & Design
Style
Books
Entertainment News
Arts
Arts in Motion
Travel
Destinations
Africa
Antarctica
Asia
Australia and Pacific
Caribbean & Bermuda
Central Ameri

In [22]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [23]:
messages_for(client)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': 'You are looking at a website titled \'Communities\' of strange, extreme life seen for first time in deep ocean\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nSkip to content\nBritish Broadcasting Corporation\nHome\nNews\nSport\nBusiness\nInnovation\nCulture\nArts\nTravel\nEarth\nAudio\nVideo\nLive\nHome\nNews\nIsrael-Gaza War\nWar in Ukraine\nUS & Canada\nUK\nUK Politics\nEngland\nN. Ireland\nN. Ireland Politics\nScotland\nScotland Politics\nWales\nWales Politics\nAfrica\nAsia\nChina\nIndia\nAustralia\nEurope\nLatin America\nMiddle East\nIn Pictures\nBBC InDepth\nBBC Verify\nSport\nBusiness\nExecutive Lounge\nTechnology of Business\nFutur

In [24]:
# And now: call the OpenAI API. You will get very familiar with this!
def summarize(url):
    website = Website(url)
    response = ollama_via_openai.chat.completions.create(
        model = "llama3.2",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [25]:
result = summarize(website_url)
print(result)

# Summary of 'Communities' Website

## Introduction

A Chinese-led research team has made a groundbreaking discovery, observing and photographing strange and extreme life forms at depths over 9km in the northwest Pacific Ocean.

## New Discoveries

* **Macellicephaloides grandicirra**: Long, thin tube worms with white, spiky features found in dense colonies, up to 30cm in length.
* **Chemosynthetic creatures**: Life forms fueled by chemicals that seep out of the ocean floor, including hydrogen sulfide and methane.

## Expedition Details

* The research team spent over 2,500km exploring trenches at depths ranging from 5,800 to 9,533m.
* They used a submersible vehicle called Fendouzhe, capable of operating at depths of more than 10km for several hours.

## Implications

* The discovery challenges long-standing assumptions about life's potential at extreme depths and pressures.
* It suggests that these communities of animals are widespread rather than rare.

## Quotes from Researchers

*

In [26]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [29]:
result2 = display_summary(website_url)
print(result2)

**Summary**

The website "Busy day at White House" discusses the recent announcement of a 25% tariff on India by US President Donald Trump. The article highlights the impact of this decision on the Indian economy and the sectors most affected.

* **Key Points**
	+ The US has imposed a 25% tariff on Indian exports, citing concerns over India's energy imports from Russia.
	+ Indian stocks have not crashed despite the announcement, showing resilience in the face of economic uncertainty.
	+ The electronics sector is expected to be hit hard by the tariffs, with a two-week reprieve given until August 1st.
	+ India has responded to the tariff by increasing its focus on diversifying energy imports and developing its own economy.

**Related Topics**

* Donald Trump's tariffs: Why India’s stock market has not crashed
* US-India trade deal
* India steps up with Quad: Target on China’s rare earth dominance
* Gold & silver price prediction today

**In Other News...**

* RAKBANK becomes first UAE conventional bank to offer crypto trading to retail customers
* HUL Q1 results: FMCG firm's profit climbs 5.97%
* Trump Tariff Impact: Which sectors are at maximum risk?
* India vs England 5th Test Match

None
